# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [ ]:
# Setup
from __future__ import division, print_function, unicode_literals, absolute_import

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Preprocessing functions
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import pickle

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [ ]:
import keras as kr

# Convolutional Neural Network

## Import Dataset

In [ ]:
################################################################################
#    Title: To Normalized
################################################################################
#    Description: 
#        This function shifts image data to be centered around 0 and bounded
#        by negative one and one.
#    
#    Parameters:
#        X    Array containing image data of range [0, 255]
# 
#    Returns:
#        Same array centered around 0 and bounded by [-1, 1]
################################################################################
def to_normalized(X):
    X = (X-128)/128
    return X

In [ ]:
################################################################################
#    Title: To One Hot
################################################################################
#    Description: 
#        This function encodes label data with one hot encoding
#    
#    Parameters:
#        X    Array of label data
# 
#    Returns:
#        One hot encoded array
################################################################################
def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

In [ ]:
################################################################################
#    Title: Get Cifar Dataset
################################################################################
#    Description: 
#        This function gets the Cifar dataset
# 
#    Returns:
#        Arrays containing training, testing, and validation data
################################################################################
def get_cifar_dataset():
    # Import data set
    from keras.datasets import cifar100
    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

    # Adjust image data to center around 0 and be bound by [-1, 1]
    x_train = to_normalized(x_train)
    x_test = to_normalized(x_test)

    # Encode labels as one hot
    y_train = to_one_hot(y_train)
    y_test  = to_one_hot(y_test)
    
    # Split training set into train and validation sets
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1, random_state=0)
    
    return (x_train, y_train), (x_val, y_val), (x_test, y_test) 

## Image Preprocessing

In [9]:
################################################################################
#    Title: Clip and Flip
################################################################################
#    Description: 
#        This clips images into the corners and center images then flips each
#        such that the resulting concatenated set is 10x as large
#    
#    Parameters:
#        X    Array of image data
#        Y    Array of label data
# 
#    Returns:
#        One hot encoded array
################################################################################
def clip_n_flip(X, Y):
    Y_conc = np.concatenate([Y, Y, Y, Y, Y, Y, Y, Y, Y, Y], axis=0)
    X_a = X[:,:28,:28]
    X_b = X[:,4:,:28]
    X_c = X[:,:28,4:]
    X_d = X[:,4:,4:]
    X_e = X[:,2:30,2:30]
    X_clip = np.concatenate([X_a,X_b,X_c,X_d,X_e], axis=0)
    X_flip = np.concatenate([X_clip, np.fliplr(X_clip)], axis=0)
    return X_flip, Y_conc

In [ ]:
################################################################################
#    Title: Read or Load Dataset
################################################################################
#    Description: 
#        This function loads the dataset and saves a pickle file if one does not
#        already exist.
#    
#    Parameters:
#        X    Array of label data
# 
#    Returns:
#        One hot encoded array
################################################################################
def read_or_load_dataset(path):
    try:
        # Load data from a file if it exists
        with open(path, "rb") as f:
            x_train = pickle.load(f)
            y_train = pickle.load(f)
            x_val = pickle.load(f)
            y_val = pickle.load(f)
            x_test = pickle.load(f)
            y_test = pickle.load(f)
    
    except StandardError:
        # Get dataset
        (x_train, y_train), (x_val, y_val), (x_test, y_test) = get_cifar_dataset
        
        # Clip and flip the images
        x_train, y_train = clip_n_flip(x_train, y_train)

        # Shuffle the images
        X_out, Y_out = shuffle(x_train, y_train, random_state=1)

        # Clip the validation and test sets to match training set
        x_val = x_val[:,2:30,2:30]
        x_test = x_test[:,2:30,2:30]
        
        # Save data to a file
        with open(path, "wb") as f:
            pickle.dump(x_train, f)
            pickle.dump(y_train, f)
            pickle.dump(x_val, f)
            pickle.dump(y_val, f)
            pickle.dump(x_test, f)
            pickle.dump(y_test, f)

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

In [12]:
dataset_path = "data/dataset.pickle"

(x_train, y_train), (x_val, y_val), (x_test, y_test) = read_or_load_dataset(dataset_path)

## Coarse Training

**Define Global Variables**

In [ ]:
# The number of coarse categories
coarse_categories = 20

# The number of fine categories
fine_categories = 100

# The threshold percentage in thresholding layer
sigma = .01

** Defining Structure**

In [30]:
################################################################################
#    Title: Independent Layers
################################################################################
#    Description: 
#        This function creates the set of layers that the coarse and each fine
#        classifier will have.
#    
#    Parameters:
#        X            The last tensor in the shared set of tensors
#        set_index    The index of the classifier section 
# 
#    Returns:
#        Last tensor prior to the softmax layer
################################################################################
def indep_layers(X, set_index):
    s = '__indep_layer_'
    net = Conv2D(128, 3, strides=1, padding='same', activation='elu', name=set_index+s+str(1))(X)
    net = Conv2D(128, 1, strides=1, padding='same', activation='elu', name=set_index+s+str(2))(net)
    net = Conv2D(128, 1, strides=1, padding='same', activation='elu', name=set_index+s+str(3))(net)
    net = MaxPooling2D((2, 2), padding='valid')(net)
    net = Dropout(0.50)(net)
    
    net = Conv2D(256, 3, strides=1, padding='same', activation='elu', name=set_index+s+str(4))(net)
    net = Conv2D(256, 1, strides=1, padding='same', activation='elu', name=set_index+s+str(5))(net)
    net = Conv2D(256, 1, strides=1, padding='same', activation='elu', name=set_index+s+str(6))(net)
    net = Dropout(0.50)(net)
    net = Flatten()(net)
    
    net = Dense(128, activation='elu', name=set_index+s+str(7))(net)
    net = Dropout(0.50)(net)

    
    return net

In [31]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, Multiply, Add, Concatenate, Dot, RepeatVector, Reshape, initializers, regularizers, constraints
from keras.models import Model


# Input Layer
input_img = Input(shape=(28, 28, 3), dtype='float32', name='main_input')

## SHARED LAYERS
net_shared = Conv2D(64, 5, strides=1, padding='same', activation='elu', name='shared_layer_1')(input_img)
net_shared = Conv2D(64, 1, strides=1, padding='same', activation='elu', name='shared_layer_2')(net_shared)
net_shared = Conv2D(64, 1, strides=1, padding='same', activation='elu', name='shared_layer_3')(net_shared)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)
net_shared = Dropout(0.50)(net_shared)

# COARSE CLASSIFIER
net = indep_layers(net_shared, '0')
output_c = Dense(fine_categories, activation='softmax')(net)

base_model = Model(inputs=input_img, outputs=output_c)

base_model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
#base_model.load_weights('data/models/base_model_70')

In [ ]:
num_epochs = 300
base_model.fit(x_train, y_train, initial_epoch=0, epochs=num_epochs, batch_size=256, validation_data=(x_val, y_val))
base_model.save_weights('data/models/base_model_'+str(num_epochs))

Train on 450000 samples, validate on 5000 samples
Epoch 1/300
133632/450000 [=======>......................] - ETA: 141s - loss: 4.2694 - acc: 0.0592

## Fine-Tuning

**Define New Thresholding Layer**

In [ ]:
from keras.engine.topology import Layer, InputSpec
from keras import backend as K
from keras.legacy import interfaces


##### THRESHOLDING LAYER IS NOT FUNCTIONAL #####
class Threshold(Layer):
    """Thresholded Rectified Linear Unit.
    This layer takes in a set of coarse probabilities and an
    index pointing to the relevant coarse probability
    
    It then thresholds that probability following:
    `f(x) = 1 for x > theta`,
    `f(x) = 0 otherwise`.
    
    Finally it multiplies that thresholded value across the 
    full input layer.
    
    # Input shape
        Arbitrary 4-D Tensor
    # Output
        An tensor with the same shape as the input but with values
        that are either all 1s or all 0s
    # Arguments
        theta: float >= 0. Threshold value.
    """

    def __init__(self, prob, index, theta=.5, **kwargs):
        super(Threshold, self).__init__(**kwargs)
        self.theta = K.cast_to_floatx(theta)
        self.index = index
        self.prob = prob

    def call(self, inputs, mask=None):
        # Get Shape of Input
        shape = inputs.get_shape().as_list()
        # Pull out coarse probability as specified by self.prob
        X = self.prob[:,self.index:self.index+1]
        # Threshold probability
        X = K.cast(K.greater(X, self.theta), K.floatx())
        # Repeat and Reshape thresholded probability to match input
        X = K.tile(X, [1, shape[1]*shape[2]*shape[3]])
        X = K.reshape(X, [-1,shape[1],shape[2],shape[3]])
        # Multiply input with thresholded probability
        X = Multiply()([X,inputs])
        return X

    def get_config(self):
        config = {'theta': float(self.theta)}
        base_config = super(Threshold, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

**Add Fine Layers**

In [ ]:
################################################################################
#    Title: Calculate Probabalistic Averaging Layer
################################################################################
#    Description: 
#        This function calculates the coarse classifier probabilities and
#        extends that to a probabalistic averaging over fine classifications
#    
#    Parameters:
#        X      The output of the coarse classifier
# 
#    Returns:
#        Coarse category predictions and probabalistic averaging layer
################################################################################
def calc_prob_ave(X):
    # Coarse to Fine mapping
    ck = Input(shape=(20, 100), dtype='float32', name='C2K')
    
    # Extend input dimension from (n,100) to (n,1,100)
    X = RepeatVector(1)(X)
    
    # Dot input, (n,1,100) and coarse to fine mapping (n,20,100) on axis 2
    X = Dot(axes=2)([ck,X])
    # Flatten result to (n,20)
    Y = Reshape([20])(X)
    
    # Dot the vectors again to get probabalistic averaging layer (n,20,100)*(n,20,1) = (n,1,100)
    X = Dot(axes=1)([ck,X])
    # Flatten the result to (n,100)
    X = Reshape([100])(X)
    return X, Y


In [ ]:
# Calculate probabalistic averaging layer and coarse probabilities
prob_ave, coarse_prob = calc_prob_ave(output_c, ck)

# Create new independent layers for each coarse category
for i in range(20):
    #thresholded = Threshold(coarse_prob,i)(net_shared) - Threshold layer not functional
    net = indep_layers(net_shared, str(i+1))
    fine_output = Dense(5, activation='softmax')(net)
    
    if i==0: output_f = fine_output
    else: output_f = Concatenate(axis=1)([output_f, fine_output])


# Apply probabalistic averaging layer to the result of the fine predictions
weighted = Multiply()([output_f, prob_ave])

**Compile new network**

In [ ]:
model = Model(inputs=[base_model.input, ck], 
              outputs=weighted)

model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Load Coarse-To-Fine Mappings**

In [ ]:
# Eventually will want to do spectral clustering for coarse to fine mappings
# currently just using coarse categories defined by cifar

##### REPLACE WITH PRE_BUILT ONE HOT ENCODER #####
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

In [ ]:
##### REPLACE WITH NP.REPEAT #####
C2K_train = np.zeros((len(x_train), len(C2K_dot), len(C2K_dot[0])))
C2K_val = np.zeros((len(x_val), len(C2K_dot), len(C2K_dot[0])))
C2K_test = np.zeros((len(x_test), len(C2K_dot), len(C2K_dot[0])))

for i in range(len(x_train)):
    C2K_train[i] = C2K_dot

for i in range(len(x_val)):
    C2K_train[i] = C2K_dot

for i in range(len(x_test)):
    C2K_train[i] = C2K_dot

**Load Coarse Weights**

In [ ]:
for layer in base_model.layers:
    # Save layer name for readability
    layer_name = layer.get_config()['name']
    
    # Set all shared layers to not be trainable
    if 'shared_layer' in layer_name:
        layer.trainable = False
    
    # Load weights for new independent layers
    if 'indep_layer' in layer_name:
        seg = layer_name.split('__')[1]
        for f_layer in model.layers:
            if seg in f_layer.get_config()['name']:
                f_layer.set_weights(layer.get_weights())

**Fit Training Data**

In [ ]:
index= 0
step = 5
stop = 50

while index < stop:
    model.fit([x_train, C2K_train], y_train, initial_epoch=index, epochs=index+step, batch_size=256, \
             validation_data=([x_val, C2K_val],y_val))
    model.save_weights('data/models/layer_'+str(index))
    index += step

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, C2K_test], y_test, batch_size=64, verbose=1)